In [ ]:
#|default_exp linkify

In [ ]:
#|export
import importlib, re, inspect
from fastcore.test import test_eq
from fastcore.utils import Path
from nbprocess.read import get_config

# Linkify Backticks
> Automatically generate links for objects in backticks

In [ ]:
#|export
def get_module(name):
    "Get a library module if it exists."
    if name.startswith('_'): return None
    try:return importlib.import_module(f'{get_config().lib_name}.{name}')
    except: return None

In [ ]:
test_eq(bool(get_module('cli')), True)
test_eq(bool(get_module('maker')), True)

In [ ]:
#|export
_re_nb_ln = re.compile(r'^# %% (\S+)')
_re_nb_mod = re.compile(r'^# AUTOGENERATED! DO NOT EDIT! File to edit: (\S+).', flags=re.MULTILINE)

def _get_baseurl() -> str:
    baseurl = get_config()['doc_host']
    return baseurl if baseurl.endswith('/') else baseurl + '/'

def _get_path(s:str, rp:re.Pattern) -> str:
    r = rp.search(s)
    if r: return Path(r.group(1)).with_suffix('.html').name
    else: return None

In [ ]:
#|hide
_tst = '# %% ../nbs/10_cli.ipynb 12'
_tst2 = '# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/02_maker.ipynb.'
test_eq(_get_path(_tst, _re_nb_ln), '10_cli.html')
test_eq(_get_path(_tst2, _re_nb_mod), '02_maker.html')

In [ ]:
#|export
def get_link(name:str) -> str:
    "Get a link to the docs from a name."
    baseurl = get_config()['doc_host']
    if '.' in name: *m,o=name.split('.')
    else: m,o=[name],None
    mod = get_module('.'.join(m))
    if mod:
        mod_src = inspect.getsource(mod)
        path = _get_path(mod_src, _re_nb_mod)
        if o:
            obj = getattr(mod, o, None)
            if not obj: return None
            if obj:
                lnum = inspect.getsourcelines(obj)[1] - 2
                path = _get_path(mod_src.splitlines()[lnum], _re_nb_ln) + f'#{o}'
        return f'{_get_baseurl()}{path}'
    else: return None
        

In [ ]:
test_eq(get_link('cli'), 'https://nbprocess.fast.ai/10_cli.html') 
test_eq(get_link('read'), 'https://nbprocess.fast.ai/01_read.html')
test_eq(get_link('read.read_nb'), 'https://nbprocess.fast.ai/01_read.html#read_nb') 
test_eq(get_link('cli.FilterDefaults'), 'https://nbprocess.fast.ai/10_cli.html#FilterDefaults')
test_eq(get_link('foo_not_exist'), None)
test_eq(get_link('read.notexist'), None)

In [ ]:
#|eval: false
from nbprocess.export import nbs_export
nbs_export()